In [2]:
import findspark

In [3]:
findspark.init('/usr/local/spark/')

In [4]:
import pyspark

In [5]:
from pyspark import SparkContext

In [ ]:
import os
os.environ['JAVA_HOME'] = "/usr/java/jre1.8.0-x64"

In [6]:
sc = SparkContext.getOrCreate()

In [ ]:
sc

In [7]:
cust_rdd = sc.textFile("customers.txt")

In [ ]:
cust_rdd.count()

In [ ]:
cust_rdd.take(2)

In [8]:
sales_rdd = sc.textFile("salestxns.tsv")

In [ ]:
sales_rdd.count()

In [9]:
sales_rdd.take(2)

["1\t43\tCamping & Hiking\t957\tDiamondback Women's Serene Classic Comfort Bi\t299.98\t1\t11599",
 '2\t48\tWater Sports\t1073\tPelican Sunstream 100 Kayak\t199.99\t1\t256']

In [10]:
def paired_rdd(str):
    line = str.split('\t')
    return line[0],line[1]

In [11]:
cust_rdd2 = cust_rdd.map(lambda line:paired_rdd(line))

In [12]:
cust_rdd2.persist()

PythonRDD[5] at RDD at PythonRDD.scala:49

In [ ]:
#cust_rdd2.cache

In [ ]:
cust_rdd2.take(5)

In [ ]:
cust_rdd2.first()

In [13]:
cust_dict = cust_rdd2.collectAsMap()

In [ ]:
type(cust_dict)

In [ ]:
for k,v in cust_dict.items():
    print(k,v)

In [ ]:
type(cust_dict)

In [ ]:
cust_dict.get('11039')

In [14]:
cust_broadcast = sc.broadcast(cust_dict)

In [ ]:
type(cust_broadcast)

In [ ]:
cust_broadcast.value

In [ ]:
type(cust_broadcast.value)

In [ ]:
cust_broadcast.value['11039']

In [ ]:
cust_broadcast.value.get('11039')

In [ ]:
sales_rdd.first()

In [15]:
def sales_rdd_pair(str):
    line = str.split('\t')
    return ((line[7],line[3],line[4],line[5]),int(line[6]))

In [16]:
sales_rdd2 = sales_rdd.map(lambda line:sales_rdd_pair(line))

In [ ]:
sales_rdd2.getNumPartitions()

In [17]:
sales_rdd2.partitionBy(10).persist()

MapPartitionsRDD[9] at mapPartitions at PythonRDD.scala:443

In [ ]:
type(sales_rdd2)

In [ ]:
sales_rdd2.first()

In [ ]:
sales_rdd2.take(30)

In [18]:
sales_rdd3 = sales_rdd2.reduceByKey(lambda x,y:x+y)

In [ ]:
sales_rdd3.take(3)

In [ ]:
type(sales_rdd3)

In [19]:
sales_rdd4 = sales_rdd3.map(lambda col:(col[0][0],col[0][1],col[0][2], float(col[0][3]), col[1], 
                                                                    round(float(col[0][3])*col[1],2)))

In [ ]:
sales_rdd4.take(3)

In [20]:
final_rdd = sales_rdd4.map(lambda col: (col[0],cust_broadcast.value.get(col[0]),col[1],col[2],col[3],col[4],col[5]))

In [ ]:
final_rdd.take(500)

In [ ]:
cust_broadcast.value.get('11039')

In [ ]:
for i in final_rdd.take(200):
    print(i)

In [21]:
final_rdd.saveAsTextFile('cust_sales')

In [ ]:
sales_rdd2.take(3)

In [90]:
sc.stop()